In [ ]:
import obspy
#filname = '4T.EL01.20220520_20220531.mseed'
filname = '4T.EL04.20220501_20220510.mseed'

st = obspy.read(filname,  format="mseed")
#nam = filname.split('4T.')[1].split('.')[0]
nam = filname.split('4T.')[1].split('.mseed')[0]
st = st.merge()
st

In [ ]:
import os
import numpy as np
import os
import io, json
import traceback
from mseed_predictor import *
import os
#from mseed_predictor import *
import os
import glob
import pandas as pd
import numpy as np
import shutil




if os.path.isdir('./Geo_MSEED/') == False:    
    os.mkdir('./Geo_MSEED/')
else:
    shutil.rmtree('./Geo_MSEED/')
    
data={}

#astart = int(filname.split('_')[0][-2:])
#aend = astart + 10
astart = st[0].stats.starttime
aend  = st[0].stats.endtime

#for ze in range(astart,aend):
while True:    
    if astart>= aend:
        break
        
    dataall = []
    stname = []
    #ti = '2022-04-' + str(ze) + 'T00:00:00.000000Z'
    #te = '2022-04-' + str(ze+1) + 'T00:00:00.000000Z'
    
    #print(astart)
    ti = obspy.UTCDateTime(astart)
    te = obspy.UTCDateTime(ti) + 86400
    print(ti, te)

    stall = st.copy()
    stall.trim(obspy.UTCDateTime(ti), obspy.UTCDateTime(te))
    # Trimming data to have the same start and end times
    tstart = obspy.UTCDateTime(stall[0].stats.starttime)
    for i in range(0,len(stall)):
        if tstart - obspy.UTCDateTime(stall[i].stats.starttime) <0:
            tstart = obspy.UTCDateTime(stall[i].stats.starttime)
    tend = obspy.UTCDateTime(stall[0].stats.endtime)
    for i in range(0,len(stall)):
        if tend - obspy.UTCDateTime(stall[i].stats.endtime) >0:
            tend = obspy.UTCDateTime(stall[i].stats.endtime)
    sttrim = stall.trim(tstart,tend)

    st2 = sttrim.copy()
    st2 = st2.merge(fill_value=0)
    st2 = st2.resample(sampling_rate=100)

    if (len(st2)==3) and (st2[0].stats.npts>=12000) and (st2[1].stats.npts>=12000) and (st2[2].stats.npts>=12000):
        st2[0].stats.channel = 'E'
        st2[1].stats.channel = 'N'
        st2[2].stats.channel = 'Z'


        if os.path.isdir('./Geo_MSEED/' + str(st2[2].stats.station)) == False:    
            os.mkdir('./Geo_MSEED/' + str(st2[2].stats.station))

        st2[0].write('./Geo_MSEED/' + str(st2[0].stats.station)  + '/' + st2[0].stats.network + '.'  + st2[0].stats.station + '.' + st2[0].stats.channel + '__' + str(st2[0].stats.starttime) + '__' + str(st2[0].stats.endtime) +'.mseed', format='MSEED')
        st2[1].write('./Geo_MSEED/' + str(st2[1].stats.station)  + '/' + st2[1].stats.network + '.'  + st2[1].stats.station + '.' + st2[1].stats.channel + '__' + str(st2[1].stats.starttime) + '__' + str(st2[1].stats.endtime) +'.mseed', format='MSEED')
        st2[2].write('./Geo_MSEED/' + str(st2[2].stats.station)  + '/' + st2[2].stats.network + '.'  + st2[2].stats.station + '.' + st2[2].stats.channel + '__' + str(st2[2].stats.starttime) + '__' + str(st2[2].stats.endtime) +'.mseed', format='MSEED')

        #filenameall.append(st2[2].stats.station + '/' + st2[2].stats.network + '.'  + st2[2].stats.station + '.' + st2[2].stats.channel + '__' + str(st2[2].stats.starttime) + '__' + str(st2[2].stats.endtime) +'.mseed')



        data[str(st2[2].stats.station)] = {}
        data[str(st2[2].stats.station)]['network'] = st2[2].stats.network
        data[str(st2[2].stats.station)]['channels'] = [st2[0].stats.channel,st2[1].stats.channel,st2[2].stats.channel]
        data[str(st2[2].stats.station)]["coord"] = [0,0,0]
        dataall.append(data)
        stname.append(str(st2[2].stats.station))





    dataall = np.array(dataall)
    with open('st_list.json', 'w') as f:
            json.dump(data,f,ensure_ascii=False)
    f.close()
    astart = obspy.UTCDateTime(astart) + 86400